Things to do:
1. [x] The get_contibution methods currently use a mix of predict_proba and predict. This should be fixed - expect probs, and calculate from there.
2. [x] Compare the runtime of get_contributions and get_contributions_slow. Check on long signal windows. If the fast version isn't faster, use the slow version (its more user friendly). **the faster script is faster, and confirmed to work**
3. [ ] Try this tutorial script with a multivariate dataset
4. [ ] Try this tutorial with unequal lengths (like japanese vowels)
5. [ ] Try this tutorial with a regression model
6. [ ] Try this tutorial with a forecasting model

Multivariate options:
https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.datasets.load_basic_motions.html#sktime.datasets.load_basic_motions
https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.datasets.load_japanese_vowels.html#sktime.datasets.load_japanese_vowels

Leaving off notes:
- To make predictions with sktime models without having to do any formatting, call predict on X_train_multi.iloc[0:1] (or a range of rows, even if just one). This works for univariate and multivariate.
- Check the cell below with a bunch of predictions to see that the current occlusion implementation has an extra index, and the code works if we don't convert to numpy

## Univariate Timeseries Classification

In [1]:
from sktime.datasets import load_arrow_head  # univariate dataset
X_train_orig, y_train_orig = load_arrow_head(split="train", return_X_y=True)
X_test_orig, y_test_orig = load_arrow_head(split="test", return_X_y=True)

X_train_orig

,dim_0
0,0 -1.963009 1 -1.957825 2 -1.95614...
1,0 -1.774571 1 -1.774036 2 -1.77658...
2,0 -1.866021 1 -1.841991 2 -1.83502...
3,0 -2.073758 1 -2.073301 2 -2.04460...
4,0 -1.746255 1 -1.741263 2 -1.72274...
5,0 -1.982806 1 -1.978861 2 -1.93733...
6,0 -2.083023 1 -2.092058 2 -2.04946...
7,0 -1.633596 1 -1.643175 2 -1.61367...
8,0 -1.717013 1 -1.728059 2 -1.68330...
9,0 -2.245295 1 -2.223850 2 -2.17187...


In [2]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier

model = ShapeletTransformClassifier()
model.fit(X_train_orig, y_train_orig)

model.score(X_test_orig, y_test_orig)

0.7314285714285714

In [25]:
import importlib
import pyreal.explainers.time_series.saliency.occlusion_saliency as oi
importlib.reload(oi)
import time
from pyreal.transformers import Transformer
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy

#class NestedToNumpy2D(Transformer):
#    def __transform__(x):
#        return x.to_frame().T

class DfToNumpy(Transformer):
    def data_transform(self,x):
        return from_nested_to_3d_numpy(x)

transformers = [DfToNumpy()]

model.predict = model.predict_proba
explainer = oi.OcclusionSaliency(model, X_train_orig, transformers=transformers, window_size=3)
explainer.fit()

X_train_orig.iloc[0].to_frame().T

start = time.time()
explanation1 = explainer.produce(X_train_orig.iloc[0])
explanation2 = explainer.produce(X_train_orig.iloc[1])

getting sklearn classes


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type array(pyobject, 1d, C)[0m
[0m[1mDuring: typing of argument at C:\Users\Ola\AppData\Local\pypoetry\Cache\virtualenvs\pyreal-BgWg7FQV-py3.8\lib\site-packages\sktime\transformations\panel\shapelet_transform.py (525)[0m
[1m
File "..\..\..\..\..\AppData\Local\pypoetry\Cache\virtualenvs\pyreal-BgWg7FQV-py3.8\lib\site-packages\sktime\transformations\panel\shapelet_transform.py", line 525:[0m
[1mdef _online_shapelet_distance(series, shapelet, sorted_indicies, position, length):
[1m    subseq = series[position : position + length]
[0m    [1m^[0m[0m


In [4]:
import pyreal.utils.visualize as vis
importlib.reload(vis)

vis.plot_timeseries_saliency(X_train_orig.iloc[0][0], explanation1[0][0:245])
vis.plot_timeseries_saliency(X_train_orig.iloc[1][0], explanation2[0][0:245])

NameError: name 'explanation1' is not defined

## Multivariate Timeseries Classification

In [5]:
from sktime.datasets import load_basic_motions #multivariate

X_train_multi, y_train_multi = load_basic_motions(split="train", return_X_y=True)
X_test_multi, y_test_multi = load_basic_motions(split="test", return_X_y=True)

In [6]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier

model_multi = ShapeletTransformClassifier()
model_multi.fit(X_train_multi, y_train_multi)

model_multi.score(X_test_multi, y_test_multi)

1.0

In [7]:
import importlib
import pyreal.explainers.time_series.saliency.occlusion_saliency as oi
importlib.reload(oi)
import time

transformers = []

model_multi.predict = model_multi.predict_proba
explainer = oi.OcclusionSaliency(model_multi, X_train_orig, window_size=3)
explainer.fit()

start = time.time()
explanation1 = explainer.produce(X_train_orig.iloc[0])
explanation2 = explainer.produce(X_train_orig.iloc[1])

getting sklearn classes


IndexError: index 1 is out of bounds for axis 0 with size 1

In [38]:
import numpy as np
from sktime.datatypes._panel._convert import from_nested_to_2d_array

sig = np.copy(X_train_orig.iloc[0][0].to_frame().T)

print(model.predict(sig))
print(model.predict(X_train_orig.iloc[0:1]))
print(model.predict(X_train_orig.iloc[0].to_frame().T))
print(model.predict(from_nested_to_3d_numpy(X_train_orig.iloc[0].to_frame().T)))
#np.argmax(model.predict(sig))

print(X_train_orig.iloc[0])

#print(sig.shape)
#print("---")
#print(X_train_orig.iloc[0].to_frame().T.shape)

[[1. 0. 0.]]
[[1. 0. 0.]]
[[1. 0. 0.]]
[[1. 0. 0.]]
dim_0    0     -1.963009
1     -1.957825
2     -1.95614...
Name: 0, dtype: object


In [12]:
from sktime.datatypes._panel._convert import from_nested_to_3d_numpy

sig = np.copy(X_train_orig.iloc[0][0].to_frame().T)

print(model_multi.predict(X_train_multi.iloc[0:1]))
print(model_multi.predict(X_train_multi.iloc[0].to_frame().T))
print(model_multi.predict(from_nested_to_3d_numpy(X_train_multi.iloc[0:1])))

#print(X_train_multi.iloc[0].to_frame().T)
#print(np.copy(X_train_multi.iloc[0].to_frame().T))
#print(from_nested_to_3d_numpy(X_train_multi.iloc[0:1]))

[[0. 0. 1. 0.]]
[[0. 0. 1. 0.]]
[[0. 0. 1. 0.]]


In [13]:
sig

array([[-1.9630089 , -1.9578249 , -1.9561449 , -1.9382889 , -1.8966569 ,
        -1.8698569 , -1.8387049 , -1.8122888 , -1.7364328 , -1.6733288 ,
        -1.6230727 , -1.5858727 , -1.5438407 , -1.4567846 , -1.3787206 ,
        -1.2924965 , -1.2169605 , -1.1089764 , -0.96868834, -0.83160026,
        -0.76030422, -0.59963213, -0.46625605, -0.30638396, -0.22684791,
        -0.08975983,  0.04137625,  0.23203876,  0.38728525,  0.41471247,
         0.51567412,  0.62614779,  0.72741025,  0.75345186,  0.78001988,
         0.83840391,  0.88817034,  0.91981996,  0.93344237,  0.9834616 ,
         1.04958   ,  1.1308921 ,  1.1898697 ,  1.2635882 ,  1.2976586 ,
         1.4139322 ,  1.4014314 ,  1.4443339 ,  1.4868475 ,  1.4448603 ,
         1.4448603 ,  1.4635131 ,  1.4635131 ,  1.4424827 ,  1.4822811 ,
         1.5221659 ,  1.5411515 ,  1.5181995 ,  1.4952875 ,  1.4739563 ,
         1.4479355 ,  1.3584794 ,  1.2685802 ,  1.2195033 ,  1.1558585 ,
         1.0848617 ,  0.97762959,  0.94645038,  0.9

## Multivariate Timeseries Classification (Variable Length)

In [ ]:
from sktime.datasets import load_japanese_vowels #multivariate variable length

X_train_multi, y_train_multi = load_japanese_vowels(split="train", return_X_y=True)
X_test_multi, y_test_multi = load_japanese_vowels(split="test", return_X_y=True)